# Infer Model Parameters for Individuals in Lung Cancer Control Group

## Show control group data

In [1]:
import os

import pandas as pd

import erlotinib as erlo


# Get data
data = erlo.DataLibrary().lung_cancer_control_group()

# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, id_key='#ID', time_key='TIME in day', biom_key='TUMOUR VOLUME in cm^3')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

**Figure 1:** To be completed

## Find MAP estimates for model parameters

In [7]:
import numpy as np
import pints
from tqdm.notebook import tqdm


# Define model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch()
model = erlo.PharmacodynamicModel(path)

# Find maximum a posteriori probability estimates (MAP)
result = pd.DataFrame(columns=['ID', 'Parameter', 'Estimate', 'Score', 'Run'])
ids = data['#ID'].unique()
for individual in tqdm(ids):
    # Get data for individual
    mask = data['#ID'] == individual
    times = data[mask]['TIME in day'].to_numpy()
    observed_volumes = data[mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = erlo.InverseProblem(model, times, observed_volumes)
    log_likelihood = pints.ConstantAndMultiplicativeGaussianLogLikelihood(problem)
    log_prior_tumour_volume = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_drug_conc = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_kappa = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_lambda_0 = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_lambda_1 = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_eta = pints.UniformLogPrior(1E-3, 1E3)  # Fixed to 1 below
    log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior = pints.ComposedLogPrior(
        log_prior_tumour_volume, 
        log_prior_drug_conc,
        log_prior_kappa,
        log_prior_lambda_0,
        log_prior_lambda_1,
        log_prior_sigma_base,
        log_prior_eta,
        log_prior_sigma_rel)
    log_posterior = pints.LogPosterior(log_likelihood, log_prior)

    # Set up optmisation controller
    optimiser = erlo.OptimisationController(log_posterior)
    
    # Fix potency kappa, and drug concentration to 0, as well as eta to 1
    # (This is the control group, and thus there is no drug)
    optimiser.fix_parameters(
        mask=[False, True, True, False, False, False,True, False],
        values=[0, 0, 1])

    # Run optimisation
    individual_result = optimiser.run()

    # Append individual result to result
    individual_result['ID'] = individual
    result = result.append(individual_result)

### Visualise optimisation results

In [8]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(result)

fig.show()

Figure 2:

## Find posterior probability distribution

In [2]:
import numpy as np
import pints
from tqdm.notebook import tqdm


# Define model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch()
model = erlo.PharmacodynamicModel(path)

# Find maximum a posteriori probability estimates (MAP)
result = pd.DataFrame(columns=['ID', 'Parameter', 'Sample', 'Iteration', 'Run'])
ids = data['#ID'].unique()
for individual in tqdm(ids):
    # Get data for individual
    mask = data['#ID'] == individual
    times = data[mask]['TIME in day'].to_numpy()
    observed_volumes = data[mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = erlo.InverseProblem(model, times, observed_volumes)
    log_likelihood = pints.ConstantAndMultiplicativeGaussianLogLikelihood(problem)
    log_prior_tumour_volume = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_drug_conc = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_kappa = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_lambda_0 = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_lambda_1 = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_eta = pints.UniformLogPrior(1E-3, 1E3)  # Fixed to 1 below
    log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior = pints.ComposedLogPrior(
        log_prior_tumour_volume, 
        log_prior_drug_conc,
        log_prior_kappa,
        log_prior_lambda_0,
        log_prior_lambda_1,
        log_prior_sigma_base,
        log_prior_eta,
        log_prior_sigma_rel)
    log_posterior = pints.LogPosterior(log_likelihood, log_prior)

    # Set up optmisation controller
    sampler = erlo.SamplingController(log_posterior)
    
    # Fix potency kappa, and drug concentration to 0, as well as eta to 1
    # (This is the control group, and thus there is no drug)
    sampler.fix_parameters(
        mask=[False, True, True, False, False, False,True, False],
        values=[0, 0, 1])

    # Run optimisation
    individual_result = sampler.run(n_iterations=100)

    # Append individual result to result
    individual_result['ID'] = individual
    result = result.append(individual_result)

Using Haario adaptive covariance MCMC
Generating 10 chains.
Running in parallel with 8 worker processess.
Iter. Eval. Accept.   Accept.   Accept.   Accept.   Accept.   Accept.   Accept.   Accept.   Accept.   Accept.   Time m:s
0     10     0         0         0         0         0         0         0         0         0         0          0:00.0
1     20     0.5       0         0.5       0.5       0.5       0.5       0.5       0.5       0.5       0.5        0:00.0
2     30     0.667     0         0.667     0.667     0.667     0.667     0.667     0.667     0.333     0.667      0:00.1
3     40     0.75      0.25      0.75      0.75      0.75      0.75      0.75      0.75      0.5       0.75       0:00.1
100   1000   0.62      0.66      0.45      0.48      0.81      0.87      0.85      0.67      0.12      0.7        0:00.5
Halting: Maximum number of iterations (100) reached.
Using Haario adaptive covariance MCMC
Generating 10 chains.
Running in parallel with 8 worker processess.
Iter. Eva

In [3]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=result, warm_up_iter=0)

fig.show()